In [1]:
!wget https://github.com/alexeygrigorev/minsearch/blob/main/minsearch/minsearch.py

--2025-06-08 00:06:33--  https://github.com/alexeygrigorev/minsearch/blob/main/minsearch/minsearch.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘minsearch.py.6’

minsearch.py.6          [ <=>                ] 187.86K  --.-KB/s    in 0.02s   

2025-06-08 00:06:33 (9.17 MB/s) - ‘minsearch.py.6’ saved [192370]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
index.fit(documents)

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
boost = {'question': 3.0, 'section': 0.5} #este campo es tres veces más importante que los demás

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5

    
)
    

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [12]:
import os
import google.generativeai as genai

In [13]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [14]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [15]:
q = 'the course has already started, can I still enroll?'

In [16]:
# 1. Define la pregunta que quieres hacer
prompt = q

# 2. Envía la pregunta al modelo y guarda la respuesta
response = model.generate_content(q)

# 3. Imprime el texto de la respuesta en pantalla
print(response.text)

It depends on the course.  Some courses allow late enrollment, while others do not.  You should check the course syllabus, the course website (if there is one), or contact the instructor or the relevant department/institution to find out if late enrollment is possible.



In [17]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [18]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\\n\n"

In [19]:
promt = prompt_template.format(question=q, context=context).strip()

In [20]:
print (promt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n
section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [21]:
query = 'When start the course?'

search_results = index.search(query=query)

context = ""
for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
prompt = prompt_template.format(question=query, context=context)


In [22]:
response = model.generate_content(prompt)

In [23]:
print(response.text)

The course will start on January 15th, 2024 at 5:00 PM.



In [24]:
def search (query):
    {'question': 3.0, 'section': 0.5}

    results = index.search(
    query=query,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5

    
    )

    return results

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [26]:
prompt = build_prompt(query, search_results)

In [27]:
def llm(prompt):

    response = model.generate_content(prompt)
    return response.text

In [28]:
query = 'how do I run kafka?'

def rag (query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
rag (query)

'The provided text gives instructions for running Kafka producers and consumers in Java and Python.  For Java,  run `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java` in the project directory. For Python, create a virtual environment using `python -m venv env`, activate it with `source env/bin/activate` (or `env/Scripts/activate` on Windows), install dependencies with `pip install -r ../requirements.txt`, and then run your Python files within that environment.  Before running the Python code, ensure all Docker images are running.\n'

In [30]:
rag ('the course has already started, can I still enroll?')

"Yes, even if you don't register, you can still submit homeworks.  However, be aware of the deadlines for final projects.\n"

In [31]:
from elasticsearch import Elasticsearch

In [32]:
es_client = Elasticsearch('http://localhost:9200')

In [33]:
es_client.info()

ObjectApiResponse({'name': '8ce628c1ff35', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'EzKY6KtgQ6qQGKergk5cdw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [34]:
index_settings = {
     "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, **index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/qsm0JRKeRbWfANnM87Z2mw] already exists')

In [35]:
documents [0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [36]:
from tqdm.auto import tqdm

In [37]:
for doc in tqdm (documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [57]:
query = 'I just discovered the course. Can I still enroll it?'

In [58]:
def elastic_search(query):

    search_query= {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, **search_query)
    
    result_docs = []
    
    for hit in response ['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [59]:
def rag (query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [60]:
rag (query)

"Yes, you can still submit homeworks even if you don't register.  However, there are deadlines for final projects, so don't delay.\n"